<a href="https://colab.research.google.com/github/zzzle96/PyTorch/blob/master/CRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import torch             
import torch.nn as nn          
import torch.nn.functional as F  
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import time

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import numpy as np

In [56]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
torch.manual_seed(777)                                      
if device =='cuda':
    torch.cuda.manual_seed_all(777) 
    

In [57]:
trans = transforms.Compose([         
    transforms.ToTensor()
])

train_data = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=trans,
                          download=True)

test_data = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=trans,
                         download=True)

In [58]:
BATCH_SIZE = 100
INPUT_SIZE = 14
HIDDEN_SIZE = 50
EPOCH = 30

In [59]:
data_loader = DataLoader(dataset = train_data, batch_size = 100, shuffle = True) 
print(data_loader)

In [47]:
class CNN_encoder(nn.Module):
    
    def __init__(self,in_f,out_f):
        super(CNN_encoder, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_f,out_f,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
    
    def forward(self, x):
      cnn_embed_seq = []
      for i in range(x.size(1)):
        c_f = self.layer1(x)    
        cnn_embed_seq.append(c_f)

      cnn_embed_seq = torch,stack(cnn_embed_seq,dim=0).transpose_(0,1)
      return cnn_embed_seq

In [80]:
class NET_LSTM_decoder(nn.Module):
  def __init__(self,in_f,out_f):
    super(NET_LSTM_decoder,self).__init__()

    #RNN layer (input_size=1, hidden_size=100, num_layer=1)
    self.rnn = nn.LSTM(input_size=in_f, hidden_size=HIDDEN_SIZE, batch_first=True, num_layers=1)
    #FC layer (input_size=100, output_size=10)
    self.FC = nn.Linear(HIDDEN_SIZE,out_f)

  def forward(self,x):
    x = x.view(BATCH_SIZE,-1,INPUT_SIZE)  #(10,1,28,28)->(10,28,28)

    out, (h_n ,c_n) = self.rnn(x)
    #print(h_n.shape)
    r_out = self.FC(h_n)

    return r_out.view(-1,10) #(1,10,10)->(10,10)

In [53]:
class LRCN(nn.Module):
  def __init__(self,cnn_feature, out_f):
    super(LRCN,self).__init__()
    self.CNN = CNN_encoder(1,cnn_feature)
    self.RNN = NET_LSTM_decoder(cnn_feature,out_f)

  def forward(self):
    return self.RNN()

In [54]:
c_f = 100
net = LRCN(data_loader,c_f).to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss().to(device)

TypeError: ignored

In [35]:
epochss = []
costss = []

total_batch = len(data_loader) 

epochs = EPOCH   #epoch:50
start = time.time()

for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data 
        print(labels)

        imgs = imgs.to(device)
        labels = labels.to(device)
        print(imgs.shape)
        optimizer.zero_grad()   
        out = LRCN(imgs)    
        #print(out.shape)
        loss = loss_func(out, labels)     
        loss.backward()         
        optimizer.step()        
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))

    epochss.append(epoch+1)
    costss.append(avg_cost)

time_elapsed = time.time()-start
print('Learning Finished!   elapsed time : {0}(sec)'.format(time_elapsed))   

tensor([7, 8, 1, 5, 7, 2, 9, 8, 8, 3, 7, 2, 0, 8, 8, 6, 9, 6, 9, 6, 8, 5, 2, 2,
        9, 8, 6, 7, 9, 7, 0, 7, 9, 2, 7, 5, 1, 2, 7, 0, 5, 9, 4, 4, 2, 6, 9, 2,
        5, 7, 4, 3, 1, 0, 2, 4, 3, 4, 1, 0, 5, 4, 0, 1, 5, 3, 2, 6, 3, 8, 0, 6,
        2, 1, 7, 2, 3, 9, 4, 2, 0, 8, 3, 1, 0, 8, 3, 0, 6, 4, 5, 7, 5, 7, 4, 0,
        1, 2, 4, 8])
torch.Size([100, 1, 28, 28])


ModuleAttributeError: ignored

In [90]:
#22222222222222222222222222222222
class NET(nn.Module):
  def __init__(self):
    super(NET,self).__init__()
    self.lstm = nn.LSTM(input_size = 14,hidden_size = 100,batch_first=True, num_layers = 1)
    self.FC = nn.Linear(100,10)

    self.conv = nn.Sequential(
         nn.Conv2d(1,28,kernel_size=3, stride=1, padding=1),
         nn.ReLU(),
         nn.MaxPool2d(2)
    )

  def forward(self, x):
    x = self.conv(x)
    x = x.view(100,-1,14)
    x, (h_n,c_n) = self.lstm(x)
    out = self.FC(h_n)

    return out.view (-1,10)
    

In [91]:
net = NET().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss().to(device)

In [95]:
epochss = []
costss = []

total_batch = len(data_loader) 

epochs = 10   #epoch:50
start = time.time()

for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data 
        #print(labels)

        imgs = imgs.to(device)
        labels = labels.to(device)
        #print(imgs.shape)
        #print(labels.shape)
        #imgs = imgs.view(-1,28,28)
        #print(imgs.shape)
        optimizer.zero_grad()   
        out = net(imgs)    
        #print(out.shape)
        loss = loss_func(out, labels)     
        loss.backward()         
        optimizer.step()        
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))

    epochss.append(epoch+1)
    costss.append(avg_cost)

time_elapsed = time.time()-start
print('Learning Finished!   elapsed time : {0}(sec)'.format(time_elapsed))   

[Epoch:1] cost = 0.1651698499917984
[Epoch:2] cost = 0.12410923093557358
[Epoch:3] cost = 0.09829071164131165
[Epoch:4] cost = 0.0818883553147316
[Epoch:5] cost = 0.06949770450592041
[Epoch:6] cost = 0.06254071742296219
[Epoch:7] cost = 0.055376119911670685
[Epoch:8] cost = 0.04751360043883324
[Epoch:9] cost = 0.044202905148267746
[Epoch:10] cost = 0.039825987070798874
Learning Finished!   elapsed time : 136.0331563949585(sec)


In [96]:
test_set = DataLoader(dataset = test_data, batch_size = 100, shuffle = True) 

In [105]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs) 

        #print(prediction.shape)
        
      
        correct_prediction = torch.argmax(prediction, 1) == label  
       
     
    accuracy_l = correct_prediction.float().mean()     
   

   
    print('Accuracy(LSTM):', accuracy_l.item())
   

Accuracy(LSTM): 0.9699999690055847
